In [2]:
import pandas as pd

In [8]:
# Players DataFrame
players_df = pd.read_csv('players_stats.csv')
players_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
#players_df

In [9]:
# Teams DataFrame
teams_df = pd.read_csv('team_stats.csv')
teams_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
#teams_df

In [10]:
# Rookie DataFrame
rookie_df = pd.read_csv('rookie_stats.csv')
rookie_df = rookie_df.iloc[1:] # Remove first row
rookie_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
#rookie_df